# L41: Lab 3 - Micro-Architectural implications of IPC

The goals of this lab are to:

- Introduce hardware performance counters (hwpmc).
- Explore micro-architectural implications of IPC.
- Gather additional data to support the writing of your first assessed lab report.

You will do this by applying PMC to analyse the behaviour of the same potted, kernel-intensive IPC benchmark used in the last lab.

## Running the benchmark

As before, you can run the benchmark using the ipc-static and ipc-dynamic commands, specifying various benchmark parameters. When the new performance-counter argument is used, additional information will be printed about the processor-level behaviour of the IPC loop. Do ensure that, as in Lab 2, you have increased the kernel’s maximum socket-buffer size.

In [ ]:
!sysctl kern.ipc.maxsockbuf=33554432

### Example benchmark commands

This command instructs the IPC benchmark to capture information on memory instructions issued when operating on a socket with a 512-byte buffer from a single thread:

In [ ]:
# Example benchmark command
print_header("Capturing info on memory instructions")

!ipc/ipc-static -vP mem -b 1048576 -i local 1thread

print_footer("Completed")

How do requested memory accesses vary across our six benchmark configurations (IPC type × operational mode)? (With some careful processing the output of the IPC benchmark can be manipulated in Python, as shown below.)

In [ ]:
import re

# Buffer sizes to compute the performance with
# Note: Perprocess resource limits prevent very large buffers with -i socket -s
BUFFER_SIZES = [512 * 2 ** exp for exp in range(0, 16)]

# Number of trials for each buffer size
NUM_TRIALS = 1

ipc_pipe_mem_read_values = []
ipc_pipe_mem_write_values = []

ipc_socket_default_mem_read_values = []
ipc_socket_default_mem_write_values = []

ipc_socket_mem_read_values = []
ipc_socket_mem_write_values = []

print_header("Capturing info on memory instructions")

for buffer_size in BUFFER_SIZES:
    print "Measuring performance for buffer size = {} bytes (max = {}) ".format(buffer_size, BUFFER_SIZES[-1])
    # -i pipe
    output = !ipc/ipc-static -i pipe -b {str(buffer_size)} -P mem 2thread

    # Convert the PMC output into JSON, to simplify post-processing
    output_json = json.loads(re.sub(r'([a-zA-Z_/0-9.]+)', r'"\1"',
        "{" + ','.join(str(e) for e in output[:-2] if e) +"}"))

    ipc_pipe_mem_read_values.append(output_json["MEM_READ"])
    ipc_pipe_mem_write_values.append(output_json["MEM_WRITE"]) 

    # -i local
    output = !ipc/ipc-static -i local -b {str(buffer_size)} -P mem 2proc

    # Convert the PMC output into JSON, to simplify post-processing
    output_json = json.loads(re.sub(r'([a-zA-Z_/0-9.]+)', r'"\1"',
        "{" + ','.join(str(e) for e in output[:-2] if e) +"}"))

    ipc_socket_default_mem_read_values.append(output_json["MEM_READ"])
    ipc_socket_default_mem_write_values.append(output_json["MEM_WRITE"]) 

    # -i local -s
    output = !ipc/ipc-static -i local -s -b {str(buffer_size)} -P mem 2proc

    # Convert the PMC output into JSON, to simplify post-processing
    output_json = json.loads(re.sub(r'([a-zA-Z_/0-9.]+)', r'"\1"',
        "{" + ','.join(str(e) for e in output[:-2] if e) +"}"))

    ipc_socket_mem_read_values.append(output_json["MEM_READ"])
    ipc_socket_mem_write_values.append(output_json["MEM_WRITE"]) 
        
print_footer("Completed")

In [ ]:
%matplotlib inline

# Reshape the list into arrays of size [len(BUFFER_SIZES), NUM_TRIALS]
ipc_pipe_mem_read = np.reshape(ipc_pipe_mem_read_values, (len(BUFFER_SIZES), 1))[:,:]
ipc_pipe_mem_write = np.reshape(ipc_pipe_mem_write_values, (len(BUFFER_SIZES), 1))[:,:]

ipc_socket_default_mem_read = np.reshape(ipc_socket_default_mem_read_values, (len(BUFFER_SIZES), 1))[:,:]
ipc_socket_default_mem_write = np.reshape(ipc_socket_default_mem_write_values, (len(BUFFER_SIZES), 1))[:,:]

ipc_socket_mem_read = np.reshape(ipc_socket_mem_read_values, (len(BUFFER_SIZES), 1))[:,:]
ipc_socket_mem_write = np.reshape(ipc_socket_mem_write_values, (len(BUFFER_SIZES), 1))[:,:]

# Convert the array of io bandwidth values into a Panda DataFrame
# this allows ploting of the median value and computation of the 
# error bars (25 and 75 percentile values)
# Note: The error bars should be small indicating that the experiment is tightly controlled
ipc_pipe_mem_read_df = pd.DataFrame(ipc_pipe_mem_read, index=BUFFER_SIZES)
ipc_pipe_mem_write_df = pd.DataFrame(ipc_pipe_mem_write, index=BUFFER_SIZES)

ipc_socket_default_mem_read_df = pd.DataFrame(ipc_socket_default_mem_read, index=BUFFER_SIZES)
ipc_socket_default_mem_write_df = pd.DataFrame(ipc_socket_default_mem_write, index=BUFFER_SIZES)

ipc_socket_mem_read_df = pd.DataFrame(ipc_socket_mem_read, index=BUFFER_SIZES)
ipc_socket_mem_write_df = pd.DataFrame(ipc_socket_mem_write, index=BUFFER_SIZES)

# Create and label the plot
fig, read = plt.subplots(2, sharex=True)

ipc_pipe_mem_read_df.median(1).plot(ax=read[0], figsize=(9,9), label="mem read", color='b', legend=True)
ipc_pipe_mem_write_df.median(1).plot(ax=read[0], figsize=(9,9),  label="mem write", color='g', linestyle='--', legend=True)

ipc_socket_default_mem_read_df.median(1).plot(ax=read[1], figsize=(9,9), label="mem read", linestyle='solid', color='b', legend=True)
ipc_socket_default_mem_write_df.median(1).plot(ax=read[1], figsize=(9,9), label="mem write", color='g', linestyle='dashed', legend=True)

ipc_socket_mem_read_df.median(1).plot(ax=read[1], figsize=(9,9), label="mem read -s", color='r', linestyle='dashdot', legend=True)
ipc_socket_mem_write_df.median(1).plot(ax=read[1], figsize=(9,9),  label="mem write -s", color='c', linestyle='dotted', legend=True)

read[0].set_title('ipc-static -i pipe 2proc')
read[0].set_xlabel('Buffer size (Bytes)')
read[0].set_ylabel('Count')
read[0].set_xscale('log')
# Plot a vertical line at 8KiB
read[0].axvline(x=8*1024, color='r', linestyle='dotted')

read[1].set_title('ipc-static -i socket 2proc')
read[1].set_xlabel('Buffer size (Bytes)')
read[1].set_ylabel('Count')
read[1].set_xscale('log')

# Display the plot and save it to a file
# (this can take a while (~30 secs) on the BeagleBone Black)
plt.savefig("ipc_static_socket_2thread_mem.pdf")